<a href="https://colab.research.google.com/github/AlbertoBassanoni/MLPNS_ABassanoni/blob/main/lessons/MLPNS23_Lesson_28_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Finals on Convolutional Neural Networks:

Un approfondimento interessante per la backpropagation dal punto di vista matematico è il blogpost al link:

http://colah.github.io/posts/2015-08-Backprop/

L'operazione di convoluzione è fondamentale nei CNN, poiché io aggiungo dei pezzi di informazioni in cui studio come interagiscono i miei elementi nel feature space sezionati dai neuroni, e mi danno un indice di covariance del mio feature space. In questo senso, il CNN viene utilizzato anche per studiare delle time series, che devono essere trasformati in una mappa bidimensionale, e rispetto ai NN classici, i CNN aggiungono informazione sulle interazioni dei pixels sulla base della loro vicinanza.

Consiglio di lettura per CNN sull'analisi delle immagini:

 https://arxiv.org/pdf/1605.07678.pdf

## Minibatch and overfitting

Vediamo quali potrebbero essere problemi di overfitting. Nel nostro codice di CNN abbiamo utilizzato un regolatore chiamato "keras.regularizers.l2", che si basa sulla metrica L2, e lo si inserisce per evitare problemi di overfitting. Un'altra operazione utile per risolvere il problema è fare dropout, ossia tagliare a caso alcuni link tra i layers di neuroni più interne. 

Il minibatch è un altro strumento che abbiamo inserito nel nostro modello all'interno della funzione "model.fit", in cui abbiamo messo "batch_size=50", e questo è stato fatto per suddividere in 50 minibatches il nostro training set e far eseguire il neural network su questi subsets che rendono la computazione più semplice e levano il problema di overfitting.

Gli hyperparameters sui minibatches e sul regularizers sono assolutamente arbitrari, non c'è un criterio analitico per trovarli, ma solo trial and error. Bisogna compensare adeguatamente un bilancio tra sufficienti dati ed eccessivo overfitting. 


## Alcuni consigli pratici:

- Un archittettura wide con molti layers tende all'overfitting, e i deep networks sono difficili da trainare;

- Il numero di epoche deve essere adeguato, in base alla loss function e anche ad altro, se non siamo attenti potrebbero volerci giorni;

- Le loss functions devono essere adeguate in base al tipo di task che deve compiere il mio NN, cioè classificazione o regressione. Per regressione tipicamente uso loss functions continue quali L1 o L2, mentre per classificazione ne uso in genere discrete, ad esempio per classificazioni binarie binary_crossentropy. In genere se ci sono dei jumps nella loss function se le activation functions sono discontinue;

- Le activation functions devono essere consistenti con le loss functions;

- Lo schema di ottimizzazione deve essere adeguato con la task e coi dati;

- Il learning rate e l'ottimizzazione deve bilanciare rapidità e accuratezza;

- Le batch side devono essere un compromesso ragionevole tra rapidità ed accuratezza, batch side più piccole sono più veloci ma danno tendenza all'overfitting;


## YOLO e R-CNN:

Abbiamo creato fino ad ora un algoritmo di object detection per il problema delle molecole. Se volessimo applicare il modello come lo abbiamo creato, il modello identifica solo oggetti tutti uguali della stessa dimensione. Più in generale, i modelli che creiamo per object detection devono avere più adattabilità per più forme. Due possibili modelli precompilati usati per object detection sono YOLO e R-CNN. 

R-CNN è la concatenazione di due modelli: uno che fa region of interest classification + uno che fa classificazione all'interno della region of interest. La prima applicazione ci metteva troppo (quasi 1 minuto per immagine), e si è passati all'implementazione di Fast-CNN, seguito anchora da Faster-CNN, che riduce di due ordini di grandezza il tempo computazionale. Sono sempre neural network di identificazione di pezzi dell'immagine.

YOLO (you only look ones) è un altro algoritmo con cui selezione delle zone di interesse e classificazione in un unico CNN. Si creano delle grid in una immagine, delle bounding boxes in cui vengono create delle class probability maps e infine si dà il detection finale.

Ci sono poi altri diversi modelli di image detection, alcuni dei quali tra i più moderni superano la performance umana! (anche se è difficile perché si apre tutto il problema della interpretabilità). 

# Physics Informed Neural Networks:

Questa è una classe interamente diversa di Neural Networks, il primo paper è del 2019, e di cosa si tratta? Fin dagli inizi, la fisica era theory driven poiché si avevano pochi dati, e bisognava sviluppare molta teoria. La falsificabilità è un criterio che serve appunto per la falsificazione della teoria. Dagli anni 80 ad oggi siamo per la maggior parte della fisica odierna in un regime data driven, ossia abbiamo un sacco di dati riguardo ai fenomeni, usiamo un sacco di inferenza statistica facendo predizioni sulla base dei dati, black-box models (ad esempio: vogliamo calcolare quanto salirà il livello del mare per effetto del global warming).

In uno spazio intermedio della fisica ci sono molti dati ma non abbastanza per prendere decisioni autonomi, quindi non possiamo trainare sufficientemente i neural networks. Tuttavia, se abbiamo una conoscenza teorica buona del nostro modello, come possiamo insegnare al nostro neural network che oltre all'analisi dei dati deve attenersi ad un certo framework teorico (aka boundary conditions)?

E' molto applicato tutto ciò nelle non linear partial differential equations, tipicamente in contesti fisici quali fluidodinamica. Diamo un esempio di una NL DE:

**$\partial_t u(t,x) + \mathcal{N}[u](t,x)=0$**

Studieremo in particolare l'equazione di Burgers, che soddisfa una legge generale di conservazione del tipo:

**$u(t)+\frac{\partial{f(u)}}{\partial{x}}$**=0

Con **$f(u)=\frac{1}{2}u^2$**. Questa equazione è molto complessa da risolvere analiticamente, e si può anche esprimere come:

**$\partial_t{u}+u\partial_x{u}-\nu \partial_{xx} u = 0$**

Tipicamente la Burgers equation si usa per shock waves formation, turbulence, weather problems, ed ho che:

- **$x$** è la posizione del fluido;
- **$t$** è il tempo;
- **$u(x,t)$** è la velocità del fluido;
- **$\nu$** è la viscosità;

Settiamo come boundary conditions:

**$u(0,x)=-sin(\pi x)$**

**$u(t,-1)=u(t,1)=0$**

Al mio neural network do soltanto le boundary conditions, e gli posso dare la velocità in qualsiasi punto compreso tra (-1,+1) (regime in cui mi fisso). Io conosco le boundary conditions, e gli do come input il calcolo di posizioni e tempi a caso tra -1 e 1 per la posizione e 0 ed 1 per il tempo. Il mio neural network deve estrapolare informazione, e tutti i neural network in genere devono fare interpolazione, tuttavia la loro guida per non andare completamente a caso è la legge teorica della Burger equation! Devo perciò inserirlo nel neural network come un constraint, e come posso darlo questo come constraint?

LA LOSS FUNCTION!!! E' l'unica cosa che dice al mio neural network se sto andando bene oppure no. Allora quello che facciamo è inserire la non linear PDE nella nostra loss function! E la loss function calcolerà i dati alle boundary conditions e non alle boundary conditions se soddisfano la Burgers equation. La Loss function sarà la somma dei residui della predizione e i residui della PDE, ossia:


**$loss = L2 + PDE = \sum_{\theta}(u_{\theta}-u)^2 + (\partial_t{u_{\theta}}+u_{\theta}\partial_x{u_{\theta}}-\nu_{\theta} \partial_{xx} u_{\theta})^2$**

Se vuoi, per approfondimento guarda il codice PINN_Burgers.ipynb che implementa un physics informed neural network per studiare l'equazione di Burgers.